# Laboratorio 3 Redes neuronales profundas
#### Orlando Cabrera 19943
#### José Hurtarte 19707

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

c:\Users\josej\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre procesamiento de datos

In [3]:
datos_mnist, info_mnist = tfds.load(name='mnist',
                                    shuffle_files = False,
                                    with_info=True, 
                                    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  50%|█████     | 2/4 [00:00<00:00,  2.28 url/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:01<00:00,  2.77 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  2.23 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:02<00:00,  1.58 url/s]


Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [4]:
entreno_mnist, prueba_mnist = datos_mnist['train'], datos_mnist['test']

In [5]:
num_obs_validacion = 0.1 * info_mnist.splits['train'].num_examples

In [6]:
num_obs_validacion = tf.cast(num_obs_validacion, tf.int64)

In [7]:
num_obs_prueba = info_mnist.splits['test'].num_examples

In [8]:
num_obs_prueba = tf.cast(num_obs_prueba, tf.int64)

In [9]:
def normalizar(imagen, etiqueta):
    imagen = tf.cast(imagen, tf.float32)
    imagen /= 255.
    return imagen, etiqueta

In [10]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)

In [11]:
datos_prueba = prueba_mnist.map(normalizar)

In [12]:
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(10000)

In [13]:
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)

In [14]:
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [15]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)

In [16]:
entradas_validacion, metas_validacion = next(iter(datos_validacion))

## 1. El ancho (tamaño de la capa escondida) del algoritmo. Intenten con un tamaño de 200.  ¿Cómo cambia la precisión de validación del modelo?  ¿Cuánto  tiempo se tardó el algoritmo en entrenar?  ¿Puede encontrar un tamaño de  capa escondida que funcione mejor?

In [69]:
tamanio_capa_escondida = 200
tamanio_salida = 10

In [72]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [73]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [74]:


modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.2724 - accuracy: 0.9213 - val_loss: 0.1249 - val_accuracy: 0.9610 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1052 - accuracy: 0.9680 - val_loss: 0.0808 - val_accuracy: 0.9753 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0702 - accuracy: 0.9785 - val_loss: 0.0636 - val_accuracy: 0.9813 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0525 - accuracy: 0.9833 - val_loss: 0.0513 - val_accuracy: 0.9837 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0411 - accuracy: 0.9873 - val_loss: 0.0404 - val_accuracy: 0.9860 - 2s/epoch - 3ms/step


Presición de validación 0.9860

In [75]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 222ms/step - loss: 0.0715 - accuracy: 0.9773


In [76]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.07. Precisión de prueba: 97.73%


Algo bastante interesante es que a pesar de que fueran solo 2 capas escondidas y al aumentarle el numero de perceptrones por cada una de estas capas escondidas aumentó la presición de la validación de 0.9852 a 0.9860 además este siempre aumentó, por lo que a pesar del aumento de perceptrones no se observa overfit. Además hay una leve mejora en la presición de prueba del 96.69% al 97.73% por lo que podemos observar un leve aumento con respecto a las 50 neuronas por capa en el modelo inicial, además obtuvo un tiempo de ejecución de la creación del modelo de 8.6 segundos

## 2. La profundidad del algoritmo.  Agreguen una capa escondida más al algoritmo. Este es un ejercicio extremadamente importante!  ¿Cómo cambia la precisión de validación?  ¿Qué hay del tiempo que se tarda en ejecutar?   Pista:  deben tener cuidado con las formas de los pesos y los sesgos.

In [24]:
tamanio_capa_escondida = 50
tamanio_salida = 10

In [25]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3a capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [33]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [27]:
modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.4086 - accuracy: 0.8794 - val_loss: 0.1970 - val_accuracy: 0.9430 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1695 - accuracy: 0.9493 - val_loss: 0.1321 - val_accuracy: 0.9597 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.1293 - accuracy: 0.9610 - val_loss: 0.1167 - val_accuracy: 0.9643 - 1s/epoch - 3ms/step
Epoch 4/5
540/540 - 1s - loss: 0.1064 - accuracy: 0.9681 - val_loss: 0.0939 - val_accuracy: 0.9728 - 1s/epoch - 3ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0912 - accuracy: 0.9724 - val_loss: 0.0879 - val_accuracy: 0.9732 - 1s/epoch - 3ms/step


Presición de validación 0.9732

In [29]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 221ms/step - loss: 0.1108 - accuracy: 0.9672


In [30]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))


Pérdida de prueba: 0.11. Precisión de prueba: 96.72%


Algo que podemos mencionar de esta red es que hay un consenso que nos dice que si tenemos una red que en total tiene más de 3 capas, se suele considerar de deep learning y cuando tiene menos se considera shallow learning. Debido a esto al aumentar capas en las redes neuronales se suele mejorar la comprensión de problemas complejos. Sin embargo en este caso pudimos observar que la presición de validación es de 0.9732 lo cual no mejora la presición de validación original lo cual también no es eficiente ya que generando mayor trabajo computacional genera resultados sub optimos.

## 3. El ancho y la profundidad del algoritmo.  Agregue cuantas capas sean necesarias para llegar a 5 capas escondidas.  Es más, ajusten el ancho del algoritmo conforme lo encuentre más conveniente.  ¿Cómo cambia la precisiónde validación? ¿Qué hay del tiempo de ejecución?

Según muchos estandares, no hay una forma exacta de definir cuales son las capas y cantidad de perceptrones optimas en cada una de las capas. Hay gente que recomienda utilizar solo 1 capa oculta con la media de la entrada y la salida o gente que recomienda que el tamaño de las capas escondidas sea 2/3 de la capa de entrada mas el tamaño de la capa de salida. Sin embargo según el siguiente artículo https://www.linkedin.com/pulse/choosing-number-hidden-layers-neurons-neural-networks-sachdev/, se suele recomendar seguir el tamaño de entrada de la red neuronal por el tamaño de salida de la red neuronal y tomar la raiz cuadrada de este numero calculado cuando deseamos utilizar una red con multiples capas escondidas. Por lo que este número en nuestro caso sería:

In [32]:
int((78*78*10)**0.5)

246

Por lo que siguiendo las directrices anteriores se generó el siguiente modelo:

In [47]:
tamanio_capa_escondida = 246
tamanio_salida = 10

In [48]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4ta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5ta capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [49]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [50]:
modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 3s - loss: 0.2579 - accuracy: 0.9217 - val_loss: 0.1341 - val_accuracy: 0.9597 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1047 - accuracy: 0.9676 - val_loss: 0.0783 - val_accuracy: 0.9767 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0740 - accuracy: 0.9772 - val_loss: 0.0668 - val_accuracy: 0.9800 - 2s/epoch - 4ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0595 - accuracy: 0.9818 - val_loss: 0.0722 - val_accuracy: 0.9783 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0521 - accuracy: 0.9839 - val_loss: 0.0597 - val_accuracy: 0.9830 - 2s/epoch - 4ms/step


Presición de validación 0.9830

In [51]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 260ms/step - loss: 0.0945 - accuracy: 0.9760


In [52]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))


Pérdida de prueba: 0.09. Precisión de prueba: 97.60%


Podemos observar que este modelo nos da una presición de validación del 0.9830 por lo que podemos decir que mejora con respecto a la presición de validación inicial del 0.9717 sin embargo esta se vuelve suboptima ya que al observar con las 2 capas ocultas de 200 perceptrones cada una en el inciso 1 nos da 97.73% de presición de prueba, y también la presición de validación es menor a la del inciso 1. Por eso a pesar de que tome valores bastante buenos entre los modelos probados es una solución suboptima. Sin embargo es una buena forma de generalizar caracteristicas de la red neuronal obteniendo buenos resultados

## 4. Experimenten con las funciones de activación.  Intenten aplicar una transformación sigmoidal a ambas capas.  La activación sigmoidal se obtiene escribiendo “sigmoid”.

In [77]:
tamanio_capa_escondida = 50
tamanio_salida = 10

In [78]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [80]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [81]:


modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.9933 - accuracy: 0.7845 - val_loss: 0.4197 - val_accuracy: 0.8953 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.3321 - accuracy: 0.9118 - val_loss: 0.2699 - val_accuracy: 0.9230 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.2436 - accuracy: 0.9312 - val_loss: 0.2156 - val_accuracy: 0.9405 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.1994 - accuracy: 0.9432 - val_loss: 0.1764 - val_accuracy: 0.9513 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.1696 - accuracy: 0.9519 - val_loss: 0.1530 - val_accuracy: 0.9577 - 1s/epoch - 2ms/step


Presición de validación 0.9577

In [82]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 238ms/step - loss: 0.1652 - accuracy: 0.9505


In [83]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.17. Precisión de prueba: 95.05%


Podemos decir de esta que tanto la presición de prueba (95.05%) y la de la validación (0.9577) con función de activación sigmoidal tiene resultados más bajos de estos campos que la que únicamente utiliza función de activación de unidades linealmente rectificadas en las capas escondidas. Por lo que podemos decir que utilizando función sigmoide de activación se obtienen peores resultados

## 5. Continúen experimentando con las funciones de activación.  Intenten aplicar un ReLu a la primera capa escondida y tanh a la segunda.  La activación tanh se obtiene escribiendo “tanh”.

In [84]:
tamanio_capa_escondida = 50
tamanio_salida = 10

In [85]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [86]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [87]:


modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.3971 - accuracy: 0.8910 - val_loss: 0.1871 - val_accuracy: 0.9470 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1617 - accuracy: 0.9524 - val_loss: 0.1278 - val_accuracy: 0.9640 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.1180 - accuracy: 0.9654 - val_loss: 0.1033 - val_accuracy: 0.9700 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0929 - accuracy: 0.9725 - val_loss: 0.0816 - val_accuracy: 0.9745 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0772 - accuracy: 0.9765 - val_loss: 0.0709 - val_accuracy: 0.9795 - 1s/epoch - 2ms/step


In [88]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 205ms/step - loss: 0.0908 - accuracy: 0.9718


In [89]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.09. Precisión de prueba: 97.18%
